# **Образец получения статистики**

In [1]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from tapi_yandex_metrika import YandexMetrikaLogsapi
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

## Ссылка на спецификации запросов Метрики

https://yandex.ru/dev/metrika/doc/api2/logs/intro.html

https://github.com/pavelmaksimov/tapi-yandex-metrika/blob/master/docs/logsapi.md

In [4]:
# токен доступа к метрике
ACCESS_TOKEN = "!!!!YOUR_TOKEN!!!!"

# номер счетчика
count_number = "61617364"

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': count_number})

In [5]:
# запишем в переменную требуемые параметры
params = {
    "fields": "ym:s:date,ym:s:clientID",
    "source": "visits",
    "date1": "2022-12-01",
    "date2": "2022-12-01"
}

In [6]:
# Check the possibility of creating a report. Via HTTP GET method.
result = client.evaluate().get(params=params)
print(result)

<TapiClient object
{   'log_request_evaluation': {   'expected_size': 1485,
                                  'log_request_sum_max_size': 10737418240,
                                  'log_request_sum_size': 10737415270,
                                  'max_possible_day_quantity': 7230582,
                                  'possible': True}}>


In [7]:
# Order a report. Via HTTP POST method.
result = client.create().post(params=params)
request_id = result["log_request"]["request_id"]
print(result)

<TapiClient object
{   'log_request': {   'attribution': 'LASTSIGN',
                       'counter_id': 61617364,
                       'date1': '2022-12-01',
                       'date2': '2022-12-01',
                       'fields': ['ym:s:date', 'ym:s:clientID'],
                       'request_id': 29897887,
                       'size': 0,
                       'source': 'visits',
                       'status': 'created'}}>


In [8]:
# получение инфо о статусе отчета
info = client.info(requestId=request_id).get()
print(info["log_request"]["status"])

processed


In [9]:
# The report can be downloaded when it is generated on the server. Via HTTP GET method.
if info["log_request"]["status"] == "processed":
    # The report can consist of several parts.
    parts = info["log_request"]["parts"]
    print("Number of parts in the report", parts)

    # The partNumber parameter specifies the number of the part of the report that you want to download.
    # Default partNumber=0
    part = client.download(requestId=request_id, partNumber=0).get()
    

else:
    print("Report not ready yet")
    
    
data = part.data    

Number of parts in the report [{'part_number': 0, 'size': 1647}]


In [11]:
# Преобразование ответа Метрики в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(data)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=0, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

,ym:s:date,ym:s:clientID
0,2022-12-01,16698851111031800941
1,2022-12-01,1669828095499892789
2,2022-12-01,1669879821792962811
3,2022-12-01,1669895320347292667
4,2022-12-01,166988992130137974


In [12]:
# OR Automatically download the report when it is prepared

client = YandexMetrikaLogsapi(
    access_token=ACCESS_TOKEN,
    default_url_params={'counterId': count_number},
    # Download the report when it will be created
    wait_report=True
)

info = client.create().post(params=params)
request_id = info["log_request"]["request_id"]

report = client.download(requestId=request_id).get()

data = report.data


print("Column names")
print(report.columns)


Column names
['ym:s:date', 'ym:s:clientID']


In [13]:
# Преобразование ответа Метрики в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(data)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=0, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

,ym:s:date,ym:s:clientID
0,2022-12-01,16698851111031800941
1,2022-12-01,1669828095499892789
2,2022-12-01,1669879821792962811
3,2022-12-01,1669895320347292667
4,2022-12-01,166988992130137974


In [14]:
# ВЫВОД ТЕКУЩИХ ЛОГОВ
result = client.allinfo().get()
print(result)

# ЦИКЛ ОЧИСТКИ ТЕКУЩИХ ЗАПРОСОВ
if len(result["requests"]) != 0:
    for i in range(0, len(result["requests"])):  # для того количества логов, которые есть в метритке
        req_id = result["requests"][i]["request_id"]   # получим номер очередного запроса
        res = client.clean(requestId=req_id).post()    # удалим этот запрос
        print(res)# распечатаем результат
else: print ("Логов на сервере нет")

<TapiClient object
{'requests': []}>
Логов на сервере нет


### Check the possibility of creating a report. Via HTTP GET method.
result = client.evaluate().get(params=params)
print(result)


### Order a report. Via HTTP POST method.
result = client.create().post(params=params)
request_id = result["log_request"]["request_id"]
print(result)


### Cancel report creation. Via HTTP POST method.
result = client.cancel(requestId=request_id).post()
print(result)


### Delete report. Via HTTP POST method.
result = client.clean(requestId=request_id).post()
print(result)


### Get information about all reports stored on the server. Via HTTP GET method.
result = client.allinfo().get()
print(result)


### Get information about a specific report. Via HTTP GET method.
result = client.info(requestId=request_id).get()
print(result)


### Download the report. Via HTTP POST method.
result = client.create().post(params=params)
request_id = result["log_request"]["request_id"]

## *Функция группировки данных по одному параметру (group_df)*

In [ ]:
def group_df(df, column):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby(column).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']
    
    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
    
    return(grouped_device)

## *Функция группировки по двум пар-м (group_df_2_param)*

In [ ]:
def group_df_2_param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby([column1, column2] ).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column1, column2, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']

    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_device)